In [7]:
import requests as rq
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
import os
import datetime
from dotenv import load_dotenv

In [8]:
# %pip install python-dotenv

In [9]:
# named_tuple = time.localtime() # get struct_time
# time_string = time.strftime("%dD%mM%YY", named_tuple)
# print(time.strftime("%dD", named_tuple))
# print(time_string)
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)
time_string = today.strftime("%dD%mM%YY")
ytime_string = yesterday.strftime("%dD%mM%YY")
print(time_string , ytime_string)


09D05M2024Y 08D05M2024Y


In [10]:
load_dotenv()
my_path = os.getcwd()
api_key = '403f16e2d699027ef9f5e9077f6842ad'
ninja_api_key = os.getenv('NINJA_API_KEY')
print(api_key)
'''
Title and abstract
Document classification codes
Date of publication
Affiliations
Citations / references
Keywords
'''

# setup the HTTP headers
header = {"Accept": "application/json", "X-ELS-APIKey": api_key}
header2 = {"Accept": "*/xml", "X-ELS-APIKey": api_key}
header3 = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"}

# Break down the variables for the search query
api_url = "https://api.elsevier.com/content/search/scopus?query="
pubyear = ['2018','2019','2020','2021','2022','2023','2024']



#create folder a day
day_folder = my_path+'\\research_csv\\'+time_string
yesterday_folder = my_path+'\\research_csv\\'+ytime_string

if not os.path.exists(day_folder):
    os.mkdir(day_folder)    

#yesterday paper info and yesterday ref info
ypaper_info = pd.DataFrame()
yref_info = pd.DataFrame()

if os.path.exists(yesterday_folder):
    if os.path.exists(yesterday_folder+'\\paper_info.csv'):
        ypaper_info = pd.read_csv(yesterday_folder+'\\paper_info.csv',index_col=0)
    if os.path.exists(yesterday_folder+'\\ref_info.csv'):
        yref_info = pd.read_csv(yesterday_folder+'\\ref_info.csv',index_col=0)

#paper next id and ref next id
paperNid = len(ypaper_info.index)
print(paperNid)


403f16e2d699027ef9f5e9077f6842ad
4200


In [11]:
dt = dict()
dt['id'] = []
dt['link']= []
dt['title']= []
dt['publicationName']= []
dt['affiliation']= []
dt['abstract'] = []
dt['publicationDate'] = []
dt['ref_count'] = []
dt['publicationYear'] = []


ref = dict()
ref['id'] = []
ref['ref_authors'] = []
ref['ref_publicationyear'] = []
ref['ref_title'] = []





def to_csv(result,dt,ref):
    oldlen = len(dt['id'])
    for id,paper in enumerate(result['search-results']['entry']):
        Nid = paperNid + id + oldlen
        dt['id'].append(Nid)
        dt['link'].append(paper['link'][2]['@href'])
        dt['title'].append(paper['dc:title'])
        dt['publicationName'].append(paper['prism:publicationName'])
        affslist = []
        if 'affiliation' in paper:
            affs = paper['affiliation']
            for aff in affs:
                affslist.append((aff['affilname'] , aff['affiliation-city'] , aff['affiliation-country']))
        dt['affiliation'].append(affslist)
        
        # print(paper['link'][2]['@href'])
        page = rq.get(paper['link'][0]['@href'],headers=header)
        result_ab = page.json()
        #get abstract
        # print("abs",result_ab)
        if 'item' in result_ab['abstracts-retrieval-response']:
            ab = result_ab['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
            #get publication date
            pubdate = result_ab['abstracts-retrieval-response']['item']['bibrecord']['head']['source']['publicationdate']
            dt['abstract'].append(ab)
            dt['publicationDate'].append(pubdate)
            dt['publicationYear'].append(pubdate['year'])
            # print(id,result_ab)
            # get reference count
            if result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail'] != None :
                refcount = result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['@refcount']
                dt['ref_count'].append(refcount)
            else:
                dt['ref_count'].append('null')
        
            #reference paper csv
            if result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail'] != None:
                allref = result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']
                for j in allref:
                    # print(Nid , j)
                    if type(j) is dict:
                        ref['id'].append(Nid)
                        if 'ref-fulltext' in j:
                            ref['ref_authors'].append(j['ref-fulltext'])
                        else:
                            ref['ref_authors'].append('null')
                        if "ref-publicationyear" in j['ref-info']:
                            ref['ref_publicationyear'].append(j['ref-info']['ref-publicationyear']['@first'])
                        else:
                            ref['ref_publicationyear'].append('null')
                        if 'ref-title' in j['ref-info']:
                            ref['ref_title'].append(j['ref-info']['ref-title']['ref-titletext'])
                        else:
                            ref['ref_title'].append('null')
        # else:
        #     dt['abstract'].append('null')
        #     dt['publicationDate'].append('null')
        #     dt['publicationYear'].append('null')
        #     dt['ref_count'].append('null')
        #     ref['id'].append(Nid)
        #     ref['ref_authors'].append('null')
        #     ref['ref_publicationyear'].append('null')
        #     ref['ref_title'].append('null')

        # break

        # df.to_csv(my_path+"\\research_csv\\paper_info\\"+"scopus_paper_"+str(id)+'.csv')

    return dt,ref


In [12]:
#publicatino year 2018 and doctype article
days_fetch = len(os.listdir('.\\research_csv'))-1
for i in range(len(pubyear)):
    search_query = '%28PUBYEAR%20%3D%20'+pubyear[i]+'%29%20AND%20%28DOCTYPE%28ar%29%29'
    itemperpage = '200'
    response = rq.get(api_url + search_query +"&count="+itemperpage  + "&start=" +str(int(itemperpage) *days_fetch) , headers = header)
    result = (response.json())
    print(result)
    dt, ref = to_csv(result,dt,ref)
    
df = pd.DataFrame.from_dict(dt)
ref_df = pd.DataFrame.from_dict(ref)

# df = pd.concat([ypaper_info , df] , join='inner' ,ignore_index=True)
# ref_df = pd.concat([yref_info , ref_df] , join='inner' , ignore_index=True)

# df.to_csv(day_folder+'\\paper_info.csv')
# ref_df.to_csv(day_folder+'\\ref_info.csv')

{'search-results': {'opensearch:totalResults': '2150043', 'opensearch:startIndex': '600', 'opensearch:itemsPerPage': '200', 'opensearch:Query': {'@role': 'request', '@searchTerms': '(PUBYEAR = 2018) AND (DOCTYPE(ar))', '@startPage': '600'}, 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/search/scopus?start=600&count=200&query=%28PUBYEAR+%3D+2018%29+AND+%28DOCTYPE%28ar%29%29', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'first', '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=200&query=%28PUBYEAR+%3D+2018%29+AND+%28DOCTYPE%28ar%29%29', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'prev', '@href': 'https://api.elsevier.com/content/search/scopus?start=400&count=200&query=%28PUBYEAR+%3D+2018%29+AND+%28DOCTYPE%28ar%29%29', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'next', '@href': 'https://api.elsevier.com/content/search/scopus?start=800&count=200&query=%28PUBYEAR+%3D+2018%29+AND+%28DOCTYPE%28ar%2

In [13]:
# df = pd.DataFrame.from_dict(dt)
# ref_df = pd.DataFrame.from_dict(ref)

# df = pd.concat([ypaper_info , df] , join='inner' ,ignore_index=True)
# ref_df = pd.concat([yref_info , ref_df] , join='inner' , ignore_index=True)

# df.to_csv(day_folder+'\\paper_info.csv')
# ref_df.to_csv(day_folder+'\\ref_info.csv')



In [14]:
# df =  pd.DataFrame.from_dict(dt)
# df.drop(df.index[0:200],axis=0,inplace=True)
# df['id'] = df['id'] - 200
# df.reset_index(inplace=True,drop=True)

# ref_df =  pd.DataFrame.from_dict(ref)
# ref_df.drop(ref_df.index[0:5978],axis=0,inplace=True)
# ref_df['id'] = ref_df['id'] - 200
# ref_df.reset_index(inplace=True,drop=True)

In [15]:
# df.to_csv(yesterday_folder+'\\paper_info.csv')
# ref_df.to_csv(yesterday_folder+'\\ref_info.csv')

In [16]:
# paper_info = pd.read_csv(my_path+'\\research_csv\\'+ time_string +'\\paper_info.csv',index_col=0)
paper_info = df
paper_info['affil_coor'] = paper_info.apply(lambda x : [] , axis=1)
coor = dict()
if os.path.exists(my_path+'\\coor.json'):
    f = open('coor.json')
    coor = json.load(f)
    f.close()
    

for i in range(0,len(paper_info.index)):
    # print(type(paper_info.loc[i,'affiliation']))
    if isinstance(paper_info.loc[i,'affiliation'] , list):
        aff = paper_info.loc[i,'affiliation']
    else: aff = eval(paper_info.loc[i,'affiliation'])
    if len(aff) > 0:
        for j in aff:
            print(j)
            city = j[1]
            country = j[2]
            if (city is not None) and (country is not None):
                coor_key = city + ' ' + country
                if coor_key not in coor:
                    api_url = 'https://api.api-ninjas.com/v1/geocoding?city='
                    response = rq.get(api_url + city + '&country=' + country, headers={'X-Api-Key': ninja_api_key})
                    resjson = response.json()
                    
                    if len(resjson) > 0:
                        coor[coor_key] = str(resjson[0]['latitude']) + " " + str(resjson[0]['longitude'])
                        paper_info.loc[i,'affil_coor'].append(coor[coor_key])
                        # print(city,country,resjson,coor)
                    else:
                        coor[coor_key] = ""
                        paper_info.loc[i,'affil_coor'].append(coor[coor_key])
                else:
                    paper_info.loc[i,'affil_coor'].append(coor[coor_key])
            else:
                paper_info.loc[i,'affil_coor'].append('')
    
with open("coor.json",'w') as coorjson:
    json.dump(coor , coorjson)


('Tokyo Institute of Technology', 'Tokyo', 'Japan')
('Vilniaus Gedimino Technikos Universitetas', 'Vilnius', 'Lithuania')
('Indian Institute of Science', 'Bengaluru', 'India')
('Sungkyunkwan University', 'Seoul', 'South Korea')
('University of Žilina', 'Zilina', 'Slovakia')
('Gottfried Wilhelm Leibniz Universität Hannover', 'Hannover', 'Germany')
('University of Bahrain', 'Zallaq', 'Bahrain')
('Yantai University', 'Yantai', 'China')
('Tsinghua University', 'Beijing', 'China')
('Częstochowa University Of Technology', 'Czestochowa', 'Poland')
('University of Prešov in Prešov', 'Presov', 'Slovakia')
('VSB – Technical University of Ostrava', 'Ostrava', 'Czech Republic')
('Università degli Studi di Milano-Bicocca', 'Milan', 'Italy')
('Université Paris 1 Panthéon-Sorbonne', 'Paris', 'France')
('Bishop Heber College, Tiruchirappalli', 'Tiruchirappalli', 'India')
('Universidad Autónoma de Madrid', 'Madrid', 'Spain')
('Southeast University', 'Nanjing', 'China')
('Sejong University', 'Seoul', 'S

In [17]:
df = pd.concat([ypaper_info , paper_info] , join='inner' ,ignore_index=True)
ref_df = pd.concat([yref_info , ref_df] , join='inner' , ignore_index=True)

df.to_csv(day_folder+'\\paper_info.csv')
ref_df.to_csv(day_folder+'\\ref_info.csv')

In [18]:
# with open("coor.json",'w') as coorjson:
#     json.dump(coor , coorjson)

In [19]:
# save_file = open("savedata2.json", "w")  
# json.dump(result_ab, save_file, indent = 6)  
# save_file.close()  


In [20]:
# %pip install pyscopus
# import pyscopus
# scopus = pyscopus.Scopus('403f16e2d699027ef9f5e9077f6842ad') 
# searches = scopus.search('(PUBYEAR = 2024)', count=25, view='STANDARD')
# print(searches.columns)

In [21]:
# print(searches['scopus_id'])

In [22]:
# pub_info = scopus.retrieve_abstract('85186412863')
# print(pub_info['abstract'])

In [23]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# import os
# # Set the download directory for the browser
# # C:\\Users\\acer\\Desktop\\code\\DataSci&DataEng\\project
# download_directory = my_path+"\\publication_recommend\\webscraping\\research_csv"
# chrome_options = webdriver.ChromeOptions()
# prefs = {"download.default_directory": download_directory}
# chrome_options.add_experimental_option("prefs", prefs)
# browser = webdriver.Chrome(options = chrome_options)
# wait = WebDriverWait(browser, 10)
# url = str(link)
# browser.get(url=url)
# time.sleep(2)
# for id,i in enumerate(result['search-results']['entry']):
#         filename = 'scopus'+str(id)
#         os.mkdir(download_directory+'\\'+filename)
#         link = i['link'][2]['@href']
#         url = str(link)
#         browser.get(url=url)
#         time.sleep(2)
#         #download research info
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module__Y4rmW']//div[@class='export-dropdown']").click()
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module___CTfk']//button[@data-testid='export-to-csv']").click()
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module___CTfk'][2]//label[@class='Checkbox-module__jE3jb Checkbox-module__zALPa']").click()
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module___CTfk'][3]//label[@class='Checkbox-module__jE3jb Checkbox-module__zALPa']").click()
#         browser.find_element(By.XPATH,"//button[@data-testid='submit-export-button']").click()
#         #dowload research reference info
#         time.sleep(3)
#         browser.find_element(By.XPATH,"//div[@id='resultsListToolbar']//label[@for='referencePageCheckbox']").click()
#         browser.find_element(By.XPATH,"//button[@id='export_references']").click()
#         time.sleep(1)
#         browser.find_element(By.XPATH,"//ul[@id='exportList']//li[@class='radio-inline valignTop'][5]").click()
#         browser.find_element(By.XPATH,"//button[(@class = 'btn btn-primary btn-sm btnEnable') and (@id = 'exportTrigger')]").click()

In [24]:
ref_df

,id,ref_authors,ref_publicationyear,ref_title
0,0,European Commission (2015) Energy Union Packag...,2015.0,NaN
1,0,European Commission (2017) White Paper on the ...,2017.0,NaN
2,1,"Barthes, Roland. 2005. The Language of Fashion...",2005.0,NaN
3,1,"Bojović, Brankica. 2010. Leksika u kriminološk...",2010.0,NaN
4,1,"Bojović, Brankica. 2013. “Pictorial Language i...",2013.0,NaN
...,...,...,...,...
201183,5599,"L.L. Fuller Policy, advocacy, and activism: on...",2016,"Policy, advocacy, and activism: on bioethicist..."
201184,5599,K.S. Ray Going beyond the data: using testimon...,2021,Going beyond the data: using testimonies to hu...
201185,5599,F. Lazaridou S. Fernando Deconstructing instit...,2022,Deconstructing institutional racism and the so...
201186,5599,"Jieman A-T, Onwumere J, Woodhead C, Stanley N,...",null,null
